In [7]:
import sklearn
import sklearn.datasets
from sklearn.feature_extraction.text import CountVectorizer
import sklearn.metrics as metrics
import sklearn.naive_bayes
from sklearn.model_selection import train_test_split

import pandas as pd
from importlib import reload
from textblob import TextBlob

import preprocessing

In [9]:
target_folder = 'Dataset'
train_data = pd.read_csv(target_folder + '/train.csv')
test_data = pd.read_csv(target_folder + '/test.csv')
sumbission_example = pd.read_csv(target_folder + '/sample_submission.csv')
sumbission_example

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.5,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0.5,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0.5,0.5,0.5,0.5,0.5,0.5
5,0001ea8717f6de06,0.5,0.5,0.5,0.5,0.5,0.5
6,00024115d4cbde0f,0.5,0.5,0.5,0.5,0.5,0.5
7,000247e83dcc1211,0.5,0.5,0.5,0.5,0.5,0.5
8,00025358d4737918,0.5,0.5,0.5,0.5,0.5,0.5
9,00026d1092fe71cc,0.5,0.5,0.5,0.5,0.5,0.5


In [10]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train_data['comment_text'])
#X_test = vectorizer.transform(test_data['comment_text'])

In [30]:
yt = train_data['toxic']
yst = train_data['severe_toxic']
yo = train_data['obscene']
yth = train_data['threat']
yi = train_data['insult']
yih = train_data['identity_hate']

In [28]:
train_data

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [11]:
y = train_data['toxic']

In [106]:
X_tr, X_te, yt_tr, yt_te, yst_tr, yst_te, yo_tr, yo_te, yth_tr, yth_te, yi_tr, yi_te, yih_tr, yih_te = train_test_split(X, yt, yst, yo, yth, yi, yih, test_size=0.33, random_state=42)

In [107]:
#clf = sklearn.linear_model.LogisticRegression(n_jobs=-1, verbose=2, solver='lbfgs', max_iter=15000)
clft = sklearn.naive_bayes.MultinomialNB(alpha=.1)
clft.fit(X_tr, yt_tr)
predt = clft.predict_proba(X_te)[:,1]
print(sklearn.metrics.roc_auc_score(yt_te, predt))

clfst = sklearn.naive_bayes.MultinomialNB(alpha=.1)
clfst.fit(X_tr, yst_tr)
predst = clfst.predict_proba(X_te)[:,1]
print(sklearn.metrics.roc_auc_score(yst_te, predst))

clfo = sklearn.naive_bayes.MultinomialNB(alpha=.07)
clfo.fit(X_tr, yo_tr)
predo = clfo.predict_proba(X_te)[:,1]
print(sklearn.metrics.roc_auc_score(yo_te, predo))

clfth = sklearn.naive_bayes.MultinomialNB(alpha=.01)
clfth.fit(X_tr, yth_tr)
predth = clfth.predict_proba(X_te)[:,1]
print(sklearn.metrics.roc_auc_score(yth_te, predth))

clfi = sklearn.naive_bayes.MultinomialNB(alpha=.07)
clfi.fit(X_tr, yi_tr)
predi = clfi.predict_proba(X_te)[:,1]
print(sklearn.metrics.roc_auc_score(yi_te, predi))

clfih = sklearn.naive_bayes.MultinomialNB(alpha=.04)
clfih.fit(X_tr, yih_tr)
predih = clfih.predict_proba(X_te)[:,1]
print(sklearn.metrics.roc_auc_score(yih_te, predih))

0.9168754452116392
0.9251312761838525
0.9199220379922546
0.8990917125836856
0.9188975418898869
0.8791104453814457


In [71]:
X_test = vectorizer.transform(test_data['comment_text'])

In [108]:
predt = clft.predict_proba(X_test)[:,1]
predst = clfst.predict_proba(X_test)[:,1]
predo = clfo.predict_proba(X_test)[:,1]
predth = clfth.predict_proba(X_test)[:,1]
predi = clfi.predict_proba(X_test)[:,1]
predih = clfih.predict_proba(X_test)[:,1]

## ToDo:
## 1. напиши норм кросс-валидацию
## 2. вставь preprocessing в CountVectorizer

In [118]:
sumbission_example['toxic']=predt
sumbission_example['severe_toxic']=predst
sumbission_example['obscene']=predo
sumbission_example['threat']=predth
sumbission_example['insult']=predi
sumbission_example['identity_hate']=predih

In [119]:
sumbission_example

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,1.000000e+00,9.228435e-02,1.000000e+00,2.609435e-04,1.000000e+00,1.000000e+00
1,0000247867823ef7,1.800142e-04,1.383801e-11,1.303000e-05,9.287653e-14,3.769283e-07,4.936908e-09
2,00013b17ad220c46,1.441527e-03,1.341302e-04,9.897555e-04,2.351234e-07,1.071069e-03,1.079062e-03
3,00017563c3f7919a,7.250882e-09,2.576021e-25,2.284567e-10,1.866372e-28,2.366799e-11,6.226241e-19
4,00017695ad8997eb,6.037160e-04,2.527510e-06,3.789642e-04,5.940089e-06,2.660280e-04,1.508048e-05
5,0001ea8717f6de06,1.128088e-04,4.176175e-15,1.172451e-05,1.318975e-12,1.293923e-05,3.348902e-10
6,00024115d4cbde0f,1.583963e-06,6.629113e-21,1.025952e-07,4.133677e-24,1.487803e-08,1.342460e-15
7,000247e83dcc1211,6.802290e-01,2.641864e-03,3.571336e-01,7.929988e-04,5.007902e-01,1.158693e-02
8,00025358d4737918,2.939997e-16,4.313048e-59,1.579575e-19,4.859079e-52,1.067081e-23,6.672206e-38
9,00026d1092fe71cc,9.701512e-13,4.252202e-33,9.324006e-14,1.600031e-26,1.868717e-15,5.934173e-23


In [122]:
sumbission_example.to_csv('nb0.csv', index=False)